<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/M6_Trabajo_Pr%C3%A1ctico_N%C2%B0_2_Parte_1_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1.1. Ejercicio 1 - Hola Mundo estilo Programacion Concurrente

1.1.1. Preguntas ejercicio 1:
a) Identifique los 3 componentes de openMP en el ejercicio 1.

Los tres componentes de openMP son directivas de compilador:


*   "#pragma omp parallel"
*   "#pragma omp single"
*   "#pragma omp master"
*   "#pragma omp for schedule(dynamic, 5)"




Bibliotecas de funciones:
omp_get_thread_num()

Variables de entorno:
env OMP_NUM_THREADS=5


b) Realice pruebas modificando la cantidad de hilos (1, 3, 7). Que conclusiones pudo determinar?.



*   Si aumento la cantidad de hilos se ve como se reparten las iteraciones entre los hilos, si tengo un solo hilo se ejecuta de manera secuencial



c) ¿Cuál es la diferencia las formas de asignación static y dynamic en la cláusula schedule?, ¿Qué sucede si utiliza dynamic en el código?



*   **STATIC:** todas las iteraciones se reparten entre los threads antes de que ejecuten el bucle. Se reparten las iteraciones equitativamente entre todos los threads
*   **DYNAMIC:** Al igual que en static se reparten todas las iteraciones entre los threads. Cuando un thread en concreto acaba las iteraciones asignadas, entonces ejecuta una de las iteraciones que estaban por ejecutar.

Si se utiliza dynamic en el codigo a la salida se ve como la distribucion de la iteraciones no es equitativa, en algunas ejecuciones el hilo 0 hizo casi todas las iteraciones, en otra un hilo hizo muchas mas iteraciones q los otros y asi. En cambio si se ejecuta con static se ve como cada hilo ejecutan la misma cantidad de iteraciones aproximadamente.





d) En el for: ¿Que sucede con el valor de j, sí se parametriza como lastprivate?



*   Si declaro la variable j como lastprivate, cada hilo tiene una copia local del dato, por lo tanto la copia global es actulizada por el hilo que ejecuta la última iteración.



In [7]:
%%writefile code.cpp

// Hola Mundo desde OpenMP usando c ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP   

int main(int argc, char* argv[]) 
{
  int i,j=0;
  std::cout<<"Inicio..."<<std::endl;

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel
  { 
    std::cout<<"Hola mundo!!!... soy el hilo "<< omp_get_thread_num()<<" de "<<  omp_get_num_threads() <<" procesadores "<< omp_get_num_procs()<< std::endl;  

    #pragma omp single
    {
      std::cout<<"Hola mundo!!!... soy el hilo "<< omp_get_thread_num()<<" uno de muchos."<< std::endl;
    }
    #pragma omp master
    {
      std::cout<<"Hola mundo!!!... soy el hilo maestro: "<< omp_get_thread_num()<< std::endl;
    }

    //#pragma omp for schedule(dynamic, 5)
    #pragma omp for lastprivate(j)
    for(i=0;i<20;i++)
    {
      j++;
      std::cout<<"Hola mundo!!!... soy el hilo "<< omp_get_thread_num()<<" itero para i. "<<i<<" actualizo j "<<j<< std::endl;
    }
  }
  // Region paralela
  //-----------------------------------------------------------------------------------------------

  std::cout<<"Fin... con j="<<j<<std::endl;
}

Writing code.cpp


In [8]:
!g++ -o hello -fopenmp code.cpp

In [11]:
%env OMP_NUM_THREADS=7
!./hello

env: OMP_NUM_THREADS=7
Inicio...
Hola mundo!!!... soy el hilo 0 de 7 procesadores 2
Hola mundo!!!... soy el hilo 0 uno de muchos.
Hola mundo!!!... soy el hilo 5 de 7 procesadores 2
Hola mundo!!!... soy el hilo 6 de 7 procesadores 2
Hola mundo!!!... soy el hilo 4 de 7 procesadores 2
Hola mundo!!!... soy el hilo 3 de 7 procesadores 2
Hola mundo!!!... soy el hilo 1 de 7 procesadores 2
Hola mundo!!!... soy el hilo 2 de 7 procesadores 2
Hola mundo!!!... soy el hilo maestro: 0
Hola mundo!!!... soy el hilo 0 itero para i. 0 actualizo j 1
Hola mundo!!!... soy el hilo 0 itero para i. 1 actualizo j 2
Hola mundo!!!... soy el hilo 0 itero para i. 2 actualizo j 3
Hola mundo!!!... soy el hilo 5 itero para i. 15 actualizo j 1
Hola mundo!!!... soy el hilo 5 itero para i. 16 actualizo j 2
Hola mundo!!!... soy el hilo 5 itero para i. 17 actualizo j 3
Hola mundo!!!... soy el hilo 6 itero para i. 18 actualizo j 1
Hola mundo!!!... soy el hilo 6 itero para i. 19 actualizo j 2
Hola mundo!!!... soy el hilo 4 

**1.2.1 Preguntas Ejercicio 2**

a) Ejecute el ejercicio 1.2.2:

- Con la variable cantidad_N para: 50, 500, 5.000.
- Con los valores hilos_ 2, 4, 6.

b) Repita la prueba (a), con ciclos 1, 10, 30. ¿Cuál fue la diferencia?

 

*   La diferencia se ve que al aumentar la cantidad de ciclos con pocos hilos, el speedup aumenta indicando la ganancia que tendria de hacerlo de forma paralela. Y si aumento los ciclos y la cantidad de hilos la eficiencia baja, en cambio si tengo pocos ciclos y bajo la cantidad de hilos la eficiencia aumenta


c) ¿Por qué el SpeedUp no mejora a medida que aumentan la cantidad de hilos?



*   Hay varias razones por la cual el speedup puede no aumentar al aumentar la cantidad de hilos, pero en este caso es debido al hardware de la maquina virtual, al generar mas hilos que procesadores se produce una competencia por los recursos lo cual no es optimo



d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?¿Porqué no llega al valor ideal?.



*   Al aumentar la cantidad_N la eficiencia aumenta acercandome al valor ideal, pero no se llega al valor ideal ya que este esta relacionado al speedup (el cual tampoco es ideal)


In [12]:
 %%writefile code_axpy.cpp
// Axpy con OpenMP usando c ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h ) gettimeofday(&tv,NULL); dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
    
    
#define TIEMPO_FIN( h ) gettimeofday(&tv,NULL); dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<<"Error en los parametros de indicar: (alfa) (Tamanio vector) (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

  std::cout<<"Valores Reales  : "<<std::endl;
  std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
  std::cout<<std::endl;
  std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
  std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
  std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
  std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


  std::cout<<std::endl;
  std::cout<<"Valores Ideal: "<<std::endl;
  TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
  std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
  std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

  std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
  std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
  std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
  std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

Overwriting code_axpy.cpp


In [13]:
!g++ -o axpy -fopenmp code_axpy.cpp

In [18]:

# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 5000#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 10#@param {type: "slider", min: 10, max: 200}
Hilos      = 1#@param {type: "slider", min: 1, max: 10}

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

env: OMP_NUM_THREADS=1
Valores Reales  : 
Tiempo TOTAL     : 1.74403 [ms]
Tiempo axpy Sec  : 0.711203 [ms]
Tiempo axpy Omp  : 0.726938 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.711203 / 0.726938 = 0.978354
Eficiencia       : SpeedUp/nro procesadores            : 0.978354 / 2 = 0.489177
Coste Sec        : nro procesadores*Tiempo             : 1 * 0.711203 = 0.711203
Coste Omp        : nro procesadores*Tiempo             : 2 * 0.726938 = 1.45388
Funcion Overhead : Coste Omp - tiempo Secuencial       : 1.45388 - 0.711203 = 0.742674

Valores Ideal: 
Tiempo axpy Sec  : 0.711203 [ms]
Tiempo axpy Omp  : 0.355601 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0.711203 / 0.355601 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo             : 1 * 0.711203 = 0.711203
Coste Omp        : nro procesadores*Tiempo             : 2 * 0.355601 = 0.711203
Funcion Overhead : Coste Omp - tiempo Secuencial 

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres